In [2]:
import pandas
from neo4j import GraphDatabase

path = "data_test.csv"
data = pandas.read_csv(path, sep=";")

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "11111111"))

# Создаем узлы "event"
with driver.session() as session:
    for row in data.itertuples(index=False):
        event_id = row[0]
        session.run("MERGE (e:Event {id_sobytiya: $event_id})", event_id=event_id)

# Создаем узлы "participant" и связи "PARTICIPATED_IN"
with driver.session() as session:
    for row in data.itertuples(index=False):
        event_id = row[0]
        participant_1 = row[1]
        participant_2 = row[2]
        session.run("""
            MERGE (p1:Participant {fio: $part_1})
            MERGE (p2:Participant {fio: $part_2})
            MERGE (p1)-[:PARTICIPATED_IN {event_id: $e_id}]->(e:Event {id_sobytiya: $e_id})
            MERGE (p2)-[:PARTICIPATED_IN {event_id: $e_id}]->(e)
        """, part_1=participant_1, part_2=participant_2, e_id=event_id)